In [1]:
import cv2
import json
import os
import numpy as np

In [3]:
# Load the JSON data from the directory
with open('C:\\Users\\ARSH\\Desktop\\ARSh\\Machine Learning\\Projects\\YOLO model\\annotations\\instances_train.json', 'r') as f:
    data = json.load(f)

In [5]:
def draw_bbox(image, bbox, color):
    """Helper function to draw bounding boxes."""
    x, y, w, h = bbox
    cv2.rectangle(image, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)

def draw_segmentation_boundary(image, segmentation, color, thickness=2):
    """Helper function to draw the boundary of the segmentation."""
    pts = np.array(segmentation[0], np.int32)  
    pts = pts.reshape((-1, 1, 2))
    cv2.polylines(image, [pts], isClosed=True, color=color, thickness=thickness)

def yolo_format_bbox(img_width, img_height, bbox):
    """Converts bounding box format to YOLO format."""
    x, y, w, h = bbox
    x_center = (x + w/2) / img_width
    y_center = (y + h/2) / img_height
    w = w / img_width
    h = h / img_height
    return x_center, y_center, w, h

In [15]:
# Directory where the train images are stored
images_directory = "C:\\Users\\ARSH\\Desktop\\ARSh\\Machine Learning\\Projects\\YOLO model\\train" 
# The new directory where the labeled images will be stored
output_directory = "C:\\Users\\ARSH\\Desktop\\ARSh\\Machine Learning\\Projects\\YOLO model\\labled_train"

In [16]:
image_dict = {image["id"]: image["file_name"] for image in data["images"]}

In [18]:
for annotation in data["annotations"]:
    image_id = annotation["image_id"]
    image_file_name = image_dict[image_id]
    image_path = os.path.join(images_directory, image_file_name)
    
    image = cv2.imread(image_path)
    image_height, image_width = image.shape[:2]

    # Draw bounding boxes on the image, and segmentation if present
    if "house_bbox" in annotation:
        draw_bbox(image, annotation["house_bbox"], (0, 255, 0))  # Green color for the house
    if "damage_bbox" in annotation:
        draw_bbox(image, annotation["damage_bbox"], (0, 0, 255))  # Red color for the damage
    if "segmentation" in annotation:
        draw_segmentation_boundary(image, annotation["segmentation"], (0, 255, 255))  # Yellow color for segmentation

    house_bbox_yolo = yolo_format_bbox(image_width, image_height, annotation["house_bbox"])
    damage_bbox_yolo = yolo_format_bbox(image_width, image_height, annotation["damage_bbox"])


    # Save the labeled image
    output_path = os.path.join(output_directory, "labeled_" + image_file_name)
    cv2.imwrite(output_path, image)

    output_txt_path = os.path.join(output_directory, os.path.splitext(image_file_name)[0] + ".txt")
    with open(output_txt_path, 'w') as txt_file:
        txt_file.write(f"0 {house_bbox_yolo[0]} {house_bbox_yolo[1]} {house_bbox_yolo[2]} {house_bbox_yolo[3]}\n")
        txt_file.write(f"1 {damage_bbox_yolo[0]} {damage_bbox_yolo[1]} {damage_bbox_yolo[2]} {damage_bbox_yolo[3]}\n")